<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_maven%2Btest_mpxj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [716]:
from numpy import mean
from numpy import std

In [717]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [718]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [719]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [720]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-maven 3.3.3-3.3.9 - maven 3.3.3-3.3.9.csv")


In [721]:
weights['Change'] = weights['Change'].map({'yes': 1, 'no': 0})

In [722]:
weights.shape

(831, 8)

In [723]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,22,10,26,237,1,73,13,0
1,2,0,18,40,0,40,5,0
2,4,0,24,71,0,27,11,0
3,2,2,16,15,0,0,3,0
4,3,0,14,7,0,0,1,0


In [724]:
X = weights.drop('Change', axis=1)
y = weights['Change']

In [725]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [726]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [727]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [728]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.7709090909090909, 0.7454545454545455, 0.6945454545454546]


In [729]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-mpxj 8.0 - mpxj 8.0.csv")

In [730]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [731]:
data.shape

(821, 8)

In [732]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,5,29,1,60,5,0
1,0,0,10,3,3,0,0,0
2,1,0,8,43,1,70,8,0
3,2,8,21,792,1,0,5,0
4,13,0,23,292,1,83,44,1


In [733]:
X = data.drop('change', axis=1)
y = data['change']

In [734]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [735]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [736]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 81.818


In [737]:
print(yhat)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [738]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[128   2]
 [ 28   7]]


In [739]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [740]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.33234567684142874


In [741]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5923076923076923


In [742]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.4437601569801833


In [743]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.82      0.98      0.90       130
           1       0.78      0.20      0.32        35

    accuracy                           0.82       165
   macro avg       0.80      0.59      0.61       165
weighted avg       0.81      0.82      0.77       165

